<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/master/modelA/Strided-CNN-A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-13 13:12:52--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-13 13:12:52 (111 MB/s) - ‘utils.py’ saved [4519/4519]



In [0]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch, lr):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        try:
          model_save_name = 'best_'+str(lr)
          path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
          torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
        except:
          print('Failed to save best model to personal google drive')

    return best_loss, best_epoch, correct

In [0]:
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

In [0]:
for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(modifiedModel=[False, True, False, False])
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc = test(epoch, best_loss, best_epoch, lr)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break
  # if in Google Colab, download your model with this
  #from google.colab import files
  #files.download("best_"+str(lr)+"pt")

Training with  0.1
Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv2d(96, 192, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): AdaptiveAvgPool2d(output_size=1)
    (14): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (conv_1_192_class): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
  (BN_96): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN_192)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.303967
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.032290

Test set: Average loss: 0.0140, Accuracy: 3488/10000 (35%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:01<00:00, 84.68it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.800798
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.768164

Test set: Average loss: 0.0136, Accuracy: 3686/10000 (37%)



...

Train Epoch: 190 [0/50000 (0%)]	Loss: 0.589230
Train Epoch: 190 [25600/50000 (51%)]	Loss: 0.591267

Test set: Average loss: 0.0054, Accuracy: 7825/10000 (78%)






Uploading file best_0.01.pt:   0%|          | 0/100 [00:00<?, ?it/s]


Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:01<00:00, 75.85it/s]

Train Epoch: 191 [0/50000 (0%)]	Loss: 0.682144
Train Epoch: 191 [25600/50000 (51%)]	Loss: 0.610682

Test set: Average loss: 0.0055, Accuracy: 7839/10000 (78%)

Train Epoch: 192 [0/50000 (0%)]	Loss: 0.617135
Train Epoch: 192 [25600/50000 (51%)]	Loss: 0.608719





Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:20<00:00, 75.85it/s]


Test set: Average loss: 0.0055, Accuracy: 7808/10000 (78%)

Train Epoch: 193 [0/50000 (0%)]	Loss: 0.450220
Train Epoch: 193 [25600/50000 (51%)]	Loss: 0.531996

Test set: Average loss: 0.0056, Accuracy: 7744/10000 (77%)

Train Epoch: 194 [0/50000 (0%)]	Loss: 0.620324
Train Epoch: 194 [25600/50000 (51%)]	Loss: 0.608635

Test set: Average loss: 0.0055, Accuracy: 7801/10000 (78%)

Train Epoch: 195 [0/50000 (0%)]	Loss: 0.605197
Train Epoch: 195 [25600/50000 (51%)]	Loss: 0.535152

Test set: Average loss: 0.0057, Accuracy: 7760/10000 (78%)

Train Epoch: 196 [0/50000 (0%)]	Loss: 0.569699
Train Epoch: 196 [25600/50000 (51%)]	Loss: 0.691515

Test set: Average loss: 0.0056, Accuracy: 7789/10000 (78%)

Train Epoch: 197 [0/50000 (0%)]	Loss: 0.564391
Train Epoch: 197 [25600/50000 (51%)]	Loss: 0.622089

Test set: Average loss: 0.0059, Accuracy: 7665/10000 (77%)

Train Epoch: 198 [0/50000 (0%)]	Loss: 0.629012
Train Epoch: 198 [25600/50000 (51%)]	Loss: 0.607308

Test set: Average loss: 0.0058, Accurac